In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/rmg_env/bin/python'
python_path = ['/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
import pandas as pd
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [3]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
import scipy
import cantera as ct

import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity

In [4]:
def find_species_phase_index(species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i

def change_species_enthalpy(species_name, dH):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(species_name)

    species = phase.species(index)
    print(f"Initial H(298) = {species.thermo.h(298)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
    #print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H(298) = {species.thermo.h(298)/1e6:.1f} kJ/mol")

# change_species_enthalpy('NH4NO3', 1e6) # 1 kJ/mol = 1e6 J/kmol
# change_species_enthalpy('NH4NO3', -1e6) # put it back for now

In [5]:
 def correct_binding_energy(species, delta_atomic_adsoprtion_energies={}):
    """
    Changes the thermo of the provided species, by applying a linear scaling relation
    to correct the adsorption energy.

    :param species: The species to modify (an RMG Species object)
    :param delta_atomic_adsoprtion_energies: a dictionary of changes in atomic adsorption energies to apply.
        mapping for each element an RMG Quantity objects with .value_si giving a value in J/mol.
    :return: None
    """
    molecule = species.molecule[0]
    # only want/need to do one resonance structure
    surface_sites = []
    for atom in molecule.atoms:
        if atom.is_surface_site():
            surface_sites.append(atom)
    normalized_bonds = {'C': 0., 'O': 0., 'N': 0., 'H': 0.}
    max_bond_order = {'C': 4., 'O': 2., 'N': 3., 'H': 1.}
    for site in surface_sites:
        numbonds = len(site.bonds)
        if numbonds == 0:
            # vanDerWaals
            pass
        else:
            assert len(site.bonds) == 1, "Each surface site can only be bonded to 1 atom"
            bonded_atom = list(site.bonds.keys())[0]
            bond = site.bonds[bonded_atom]
            if bond.is_single():
                bond_order = 1.
            elif bond.is_double():
                bond_order = 2.
            elif bond.is_triple():
                bond_order = 3.
            elif bond.is_quadruple():
                bond_order = 4.
            else:
                raise NotImplementedError("Unsupported bond order {0} for binding energy "
                                          "correction.".format(bond.order))

            normalized_bonds[bonded_atom.symbol] += bond_order / max_bond_order[bonded_atom.symbol]


    # now edit the adsorptionThermo using LSR
    change_in_binding_energy = 0.0
    for element in delta_atomic_adsoprtion_energies.keys():
        change_in_binding_energy += delta_atomic_adsoprtion_energies[element].value_si * normalized_bonds[element]
    if change_in_binding_energy != 0.0:
        print(f"Applying LSR correction to {species.label}:")
        change_species_enthalpy(species.label, change_in_binding_energy*1000)

In [8]:
# Create Gas Solution and Surface Interface from cti file
cti_file = "./chem_annotated_Pt_large.cti" # path to cti file
gas = ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])
# species_dict = rmgpy.data.base.Database().get_species('species_dictionary_Pt_large.txt',resonance=False)
#species_dict = rmgpy.data.base.Database().get_species('../RMG-model/chemkin/species_dictionary.txt',resonance=False)

In [22]:
print(len(surf.delta_enthalpy), len(surf.reactions()))

297 297


In [17]:
gas.species()

[<Species HAN>,
 <Species NH4NO3>,
 <Species Ne>,
 <Species NH3(2)>,
 <Species NH2OH(3)>,
 <Species HNO3(4)>,
 <Species CH3OH(5)>,
 <Species H2O(6)>,
 <Species N2(7)>,
 <Species O2(8)>,
 <Species NO2(9)>,
 <Species NO(10)>,
 <Species N2O(11)>,
 <Species CO2(12)>,
 <Species H2(13)>,
 <Species CO(14)>,
 <Species CH4(15)>,
 <Species C2H6(16)>,
 <Species CH2O(17)>,
 <Species CH3(18)>,
 <Species C3H8(19)>,
 <Species H(20)>,
 <Species C2H5(21)>,
 <Species HCO(22)>,
 <Species CH3CHO(23)>,
 <Species OH(24)>,
 <Species C2H4(25)>,
 <Species HONO(32)>,
 <Species O(45)>,
 <Species Ar(46)>,
 <Species HO2(48)>,
 <Species H2O2(49)>,
 <Species HOCO(50)>,
 <Species CH2(51)>,
 <Species CH2(S)(52)>,
 <Species CH(53)>,
 <Species CH2OH(54)>,
 <Species CH3O(55)>,
 <Species HCOH(56)>,
 <Species CH3OO(57)>,
 <Species CH2CO(58)>,
 <Species C2H3(59)>,
 <Species C(60)>,
 <Species C2H2(61)>,
 <Species C2H(62)>,
 <Species CH3OOH(63)>,
 <Species CH2OOH(64)>,
 <Species HOCH2O(65)>,
 <Species HOCHO(66)>,
 <Species C2